In [1]:
# usual imports
import os
import numpy as np
import matplotlib.pyplot as plt
from rail.core.utils import RAILDIR
#from rail.estimation.algos.bpz_version.utils import RAIL_BPZ_DIR
from rail.pipelines.examples.survey_nonuniformity.survey_nonuniformity import SurveyNonuniformDegraderPipeline
from rail.core import common_params
from rail.pipelines.utils.name_factory import NameFactory, DataType, CatalogType, ModelType, PdfType
import qp
import ceci

Imported rail.hub
Imported rail.astro_tools
Imported rail.core
Imported rail.stages
Imported rail.bpz
Imported rail.cmnn
Imported rail.delight
Failed to import rail.dsps because: You need to have the SPS_HOME environment variable
Imported rail.flexzboost
Imported rail.gpz
Imported rail.pipelines
Failed to import rail.pzflow because: No module named 'rail.estimation.algos.pzflow'
Imported rail.sklearn
Imported rail.som
Attached 12 base classes and 53 fully formed stages to rail.stages


### Define the bands, magnitude limits, and put them into a dictionary

In [8]:
bands = 'grizy'
maglims = [27.79, 29.04, 29.06, 28.62, 27.98, 27.05]
maglim_dict={}
for band,limx in zip(bands, maglims):
    maglim_dict[f"mag_{band}_lsst"] = limx

### Setting common parameters to all the stages in the estimation


In [10]:
common_params.set_param_defaults(
    bands=[f'mag_{band}_lsst' for band in bands], # bands
    err_bands=[f'mag_err_{band}_lsst' for band in bands], # uncertainty bands
    nondetect_val=np.nan,
    #ref_band='HSCi_cmodel_dered',
    redshift_col='redshift',
    mag_limits=maglim_dict,
    #zmax=6.0, # set zmax to 6 for HSC
)

### Set up the pipeline

In [11]:
pipe = SurveyNonuniformDegraderPipeline()

Inserting handle into data store.  model: /global/u2/q/qhang/desc/rail_base/src/rail/examples_data/goldenspike_data/data/pretrained_flow.pkl, flow_engine_train


2023-08-03 15:58:44.859000: E external/xla/xla/stream_executor/cuda/cuda_dnn.cc:445] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2023-08-03 15:58:44.859049: E external/xla/xla/stream_executor/cuda/cuda_dnn.cc:449] Memory usage: 3276800 bytes free, 42297524224 bytes total.


XlaRuntimeError: FAILED_PRECONDITION: DNN library initialization failed. Look at the errors above for more details.

In [12]:
namer = NameFactory()

### Define parameters for each stage

In [ ]:
# dictionary used by ObsCondition
# first define a set of input map directories:

base_path = "/pscratch/sd/q/qhang/rubin_baseline_v2/MAF-1year/"

# nside of these maps:
nside=128

# seeing maps:
seeing_u = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_u_and_nightlt365_HEAL.fits"
seeing_g = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_g_and_nightlt365_HEAL.fits"
seeing_r = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_r_and_nightlt365_HEAL.fits"
seeing_i = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_i_and_nightlt365_HEAL.fits"
seeing_z = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_z_and_nightlt365_HEAL.fits"
seeing_y = base_path + "baseline_v2_0_10yrs_Median_seeingFwhmEff_y_and_nightlt365_HEAL.fits"

# coadd depth maps:
coaddm5_u = base_path + "baseline_v2_0_10yrs_CoaddM5_u_and_nightlt365_HEAL.fits"
coaddm5_g = base_path + "baseline_v2_0_10yrs_CoaddM5_g_and_nightlt365_HEAL.fits"
coaddm5_r = base_path + "baseline_v2_0_10yrs_CoaddM5_r_and_nightlt365_HEAL.fits"
coaddm5_i = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
coaddm5_z = base_path + "baseline_v2_0_10yrs_CoaddM5_z_and_nightlt365_HEAL.fits"
coaddm5_y = base_path + "baseline_v2_0_10yrs_CoaddM5_y_and_nightlt365_HEAL.fits"

# here we will set the observing year and number of visits per year to 1, because we are supplying coadd depth

# mask:
maskdir = base_path + "../wfd_footprint_nvisitcut_500_nside_128.fits"

# weight: for now we supply uniform weight

# choose the systematic map to examine, here we choose the combined depth:
sys_to_check = base_path + "baseline_v2_0_10yrs_CoaddM5_i_and_nightlt365_HEAL.fits"
sys = "CoaddM5"

map_dict={
        "theta": {
            "u": seeing_u,
            "g": seeing_g,
            "r": seeing_r,
            "i": seeing_i,
            "z": seeing_z,
            "y": seeing_y,
        },
        "m5": {
            "u": coaddm5_u,
            "g": coaddm5_g,
            "r": coaddm5_r,
            "i": coaddm5_i,
            "z": coaddm5_z,
            "y": coaddm5_y,
        },
        "nYrObs": 1.,
        "nVisYr": {
            "u": 1.,
            "g": 1.,
            "r": 1.,
            "i": 1.,
            "z": 1.,
            "y": 1., 
        },
        "sigLim": 10,
        "ndFlag": np.nan,
        "extendedType": "auto",
        "majorCol": "major",
        "minorCol": "minor",
        "decorrelate": True,
        "highSNR": False,
        "renameDict": band_dict,  
    }

In [ ]:
pipe.flow_engine_train.config.update(n_samples=20)
pipe.obs_condition.config.update(map_dict=map_dict, nside=nside, mask = maskdir, weight = "",)
pipe.deredden.config.update(dustmap_dir=".",)# where to save the dust map downloaded
pipe.estimate_bpz.config.update(spec_groupname = "", redshift_colname = 'redshift', prior_band='i',)

In [ ]:
pipe_info = pipe.initialize(input_dict, dict(output_dir='.', log_dir='.', resume=True), None)

### Save the pipeline into a yml file

In [ ]:
pipe.save('tmp_survey_nonuniformity.yml')

### run pipeline

In [ ]:
import ceci
pr = ceci.Pipeline.read('tmp_survey_nonuniformity.yml')

In [ ]:
pr.run()

# Redshift distribution in each tomographic bins at different depth:

In [ ]:
# tomographic bin n(z) at different depth:

# load the specific systematic map & mask to check correlation
mapin = hp.read_map(sys_to_check)
mask = hp.read_map(maskdir)
# quantile contains pixel indices, and meanv is the mean value of the systematic maps in each quantile
quantile, meanv = split_sys_map_quantiles(mapin, mask, nquantiles=nquantiles)

In [ ]:
# compute simple summary statistic
nzstat_summary_split={}

for jj in range(npzbins):
    nzstat_summary_split["tomo-%d"%(jj+1)]={}
    
    ind0 = data_degraded_gold_tomo["tomo"] == (jj+1)

    for ii in range(nquantiles):
        ind = np.in1d(data_degraded_gold_tomo["pixels"], quantile[ii])
        ind = ind * ind0
        usecat = data_degraded_gold_tomo.loc[ind, :]
        # now for each tomographic bin, return redshift distribution:
        nzstat_summary_split["tomo-%d"%(jj+1)][ii] = compute_nzstats(usecat, z_col, 
                                                                     zgrid=zgrid, nbootstrap=nbootstrap)

    # compute the tot nz, meanz, sigmaz:
    nzstat_summary_tot["tomo-%d"%(jj+1)] = compute_nzstats(data_degraded_gold_tomo, z_col, 
                                                           zgrid=zgrid, nbootstrap=nbootstrap)

In [ ]:
# write to file:
outroot = savedir + "test-pz-with-i-band-coadd-Y1.yml"
write_evaluation_results(outroot, meanv, nzstat_summary_split, nzstat_summary_tot)

In [ ]:
# Show results in a plot:
fig,axarr=plt.subplots(3,npzbins,figsize=[15,10],gridspec_kw={'height_ratios': [3, 1, 1]})

## Top row: n(z) for each tomographic bin for each depth group
## Middle row: change in meanz as a function of depth with bootstrap errors
## Bottom row: change in sigmaz as a function of depth with boostrap errors

for ii in range(npzbins):
    
    # top row
    plt.sca(axarr[0,ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        nz = stat_summary_split["tomo-%d"%(ii+1)][q][0]
        plt.plot(nz[:,0], nz[:,1]/np.sum(nz[:,1])/(nz[1,0]-nz[0,0]), 
                color=cmap(colorlab))
    plt.text(0.6, 3.5, "tomo-%d"%(ii+1))
    plt.yticks([])
    plt.xlabel("$z$")
    
    # middle row
    plt.sca(axarr[1, ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        
        meanz = stat_summary_split["tomo-%d"%(ii+1)][q][1]
        meanztot = stat_summary_tot["tomo-%d"%(ii+1)][q][1]
        
        plt.errorbar(meanv[q], meanz[0], yerr=meanz[1],fmt='o',
                    color=cmap(colorlab))
    #dz = 0.005*(1+meanztot[0])
    plt.plot(meanv, np.ones(len(meanv))*meanztot[0], 'k-', alpha=0.5)
    #plt.fill_between([meanv[0], meanv[-1]], [-dz, -dz], 
                    #[dz, dz],color='k',alpha=0.2)
    if ii==0:
        plt.ylabel("$\\langle z\\rangle$")
    if ii>0:
        plt.yticks([])
    #plt.xlabel(sys)
    #plt.ylim([-0.015,0.015])
    #plt.xlim([24.6, 25.7])
    
    # bottom row
    plt.sca(axarr[2, ii])
    for q in range(nquantiles):
        colorlab = q/(nquantiles*1.2)
        
        sigmaz = stat_summary_split["tomo-%d"%(ii+1)][q][2]
        sigmaztot = stat_summary_tot["tomo-%d"%(ii+1)][q][2]
        
        plt.errorbar(meanv[q], sigmaz[0], yerr=sigmaz[1],fmt='o',
                    color=cmap(colorlab))
    #dz = 0.005
    plt.plot(meanv, np.ones(len(meanv))*sigmaztot[0], 'k-', alpha=0.5)
    #plt.fill_between([meanv[0], meanv[-1]], [-dz, -dz], 
                    #[dz, dz],color='k',alpha=0.2)
    if ii==0:
        plt.ylabel("$\\sigma_z$")
    if ii>0:
        plt.yticks([])
    plt.xlabel(sys)
    #plt.ylim([-0.015,0.015])

plt.tight_layout()
plt.saveifg(savedir + 'fig.png', bbox_inches='tight')

In [ ]:
###tests